In [1]:
#setup

import json
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import os
from copy import deepcopy
# %matplotlib inline
# %matplotlib widget
plt.rcParams['figure.figsize'] = [10, 5]


In [2]:
prefix = "mdpvK"
jsonFileName = prefix + ".json"
jsonDict = {}

epsilon = 1 # random vs policy actions while training
epsilons = [0,1]
convN = 10 # evaluates performance after convN episodes. Evaluates if converged after 10 performance evals
discount = 0.6 # short term vs long term reward
syncBackups = 1 # model backups between each sync
stochasticPolicy = True # sample from softmax instead of using argmax
envSeed = 42
alpha = 1 # step size
fedP = 2 # num agents
fedPs = [2, 4, 6, 8, 10]
syncBackupValues = [10000, 1000, 100, 10, 1]

# jupyter nbconvert --no-input --to pdf graphing.ipynb

print("epsilons " + str(epsilons))
print("convN " + str(convN))
print("dicount " + str(discount))
print("syncBackups " + str(syncBackups))
print("stochasticPolicy " + str(stochasticPolicy))
print("envSeed " + str(envSeed))
print("alpha " + str(alpha))
print("fedPs " + str(fedPs))

epsilons [0, 1]
convN 10
dicount 0.6
syncBackups 1
stochasticPolicy True
envSeed 42
alpha 1
fedPs [2, 4, 6, 8, 10]


In [3]:
if os.path.isfile(jsonFileName):
    jsonFile = open(jsonFileName)
    jsonDict = json.load(jsonFile)
    jsonFile.close()


In [4]:
key = list(jsonDict.keys())[0]
print(key)
keyList = key[1:-1].split(", ")
print(keyList)
itemKeys = list(jsonDict[key].keys())
print(itemKeys)

print("learned ", jsonDict[list(jsonDict)[0]]["endScore"])
print("bell ", jsonDict[list(jsonDict)[0]]["endScoreBell"])
print("random ", jsonDict[list(jsonDict)[0]]["endScoreRand"])

(10, 0.1, 0.6, 1.0, 256, 880, True, 3701, 2)
['10', '0.1', '0.6', '1.0', '256', '880', 'True', '3701', '2']
['sims', 'diffs', 'rewards', 'endScoreStoch', 'avgRewArr', 'comment', 'avgRew', 'episodes', 'endScoreBell', 'endScoreRand', 'aggs', 'backups', 'epsToBackup', 'endScoreStochBell', 'endScore']
learned  3898.6047529358507
bell  3913.432567588432
random  1281.841492486209


In [ ]:
# jsonDictKey = str((convN, alpha, discount, epsilon, fedP, syncBackups, stochasticPolicy, envSeed, trial))
csvList = []
labels = ["convN", "alpha", "discount", "epsilon", "fedP", "syncBackups", "stochasticPolicy", "envSeed", "trial"]
itemKeys = ['sims', 'endScoreStoch', 'comment', 'avgRew', 'episodes', 'endScoreBell', 'endScoreRand', 'aggs', 'backups', 'endScoreStochBell', 'endScore']
arrayKeys = ['diffs', 'rewards', 'avgRewArr', 'epsToBackup']
summaryKeys = ['finalDiff', "finalDiffAvg", "threshDiff", "threshRew", "threshEp", "threshBack"]

csvList.append(labels + itemKeys + summaryKeys)
thresh = 0.1

for key, value in jsonDict.items():
    line = key[1:-1].split(", ")
    for itemKey in itemKeys:
        if itemKey not in value:
            line.append(0)
        else:
            line.append(value[itemKey])

    # finalDiff
    line.append(value["diffs"][-1])
    # finalDiffAvg
    line.append(np.mean(value["diffs"][-convN:]))
    # find idx where diffs[i] < 0.1
    idx = np.nonzero(np.array(value["diffs"]) ** 2 < thresh)
    # print(idx, idx[0].shape[0])
    if idx[0].shape[0] == 0:
        idx = int(value["episodes"]) - 1
    else:
        idx = idx[0][0] * convN

    # print(idx)
    # print(len(value["diffs"]))
    # print(len(value["epsToBackup"]))

    # threshDiff
    line.append(value["diffs"][int(idx / convN)])
    # print(value["diffs"][0])
    # threshRew
    line.append(value["avgRewArr"][int(idx / convN)])
    # threshEp
    line.append(idx)
    
    # threshBack
    if idx+convN >= len(value["epsToBackup"]):
        line.append(value['backups'])
    else:
        line.append(value["epsToBackup"][idx + convN])
        
    csvList.append(line)

np.savetxt(prefix + ".csv", csvList, delimiter=",", fmt="%s")


In [22]:
# jsonDictKey = str((convN, alpha, discount, epsilon, fedP, syncBackups, stochasticPolicy, envSeed, trial))
csvList = []
labels = ["convN", "alpha", "discount", "epsilon", "fedP", "syncBackups", "stochasticPolicy", "envSeed", "trial"]
itemKeys = ['sims', 'endScoreStoch', 'comment', 'avgRew', 'episodes', 'endScoreBell', 'endScoreRand', 'aggs', 'backups', 'endScoreStochBell', 'endScore']
arrayKeys = ['diffs', 'avgRewArr', 'epsToBackup']

csvList.append(labels + itemKeys + arrayKeys)

for key, value in jsonDict.items():
    line = key[1:-1].split(", ")
    for itemKey in itemKeys:
        if itemKey not in value:
            line.append(0)
        else:
            line.append(value[itemKey])
            
    # hack to add step zero distance. Since q table is initialized to be a zero array it is a constant for all runs
    # lineTemp = deepcopy(line)
    # lineTemp.append(3.7055958469869537) # distance(0, bellman)
    # lineTemp.append(0) # 0 reward
    # lineTemp.append(0) # 0 backups
    # csvList.append(lineTemp)

    for i in range(len(value["diffs"])):
        # finalDiff, threshDiff, threshRew, threshEp, threshBack
        lineTemp = deepcopy(line)

        # find idx where diffs[i] < 0.1

        # print(idx)
        # print(len(value["diffs"]))
        # print(len(value["epsToBackup"]))

        lineTemp.append(value["diffs"][i])
        # print(value["diffs"][0])
        lineTemp.append(value["avgRewArr"][i])
        # lineTemp.append(idx)
        if (i+1) * convN >= len(value["epsToBackup"]):
            lineTemp.append(value['backups'])
        else:
            lineTemp.append(value["epsToBackup"][(i+1) * convN])
        csvList.append(lineTemp)
        
np.savetxt(prefix + "Traj.csv", csvList, delimiter=",", fmt="%s")

In [ ]:
# read file
data = {}
for epsilon in epsilons:
    for syncBackups in syncBackupValues:
        alpha = 1

        epsSin = []
        epsFed = {}
        scoresSin = []
        scoresFed = {}
        scoresBell = []
        scoresStochSin = []
        scoresStochFed = {}
        scoresStochBell = []
        scoresRand = []
        diffsSin = []
        diffsFed = {}
        alphas = []
    
    
    
        for a in range(10):
            for fedP in fedPs:
                if not fedP in epsFed:
                    epsFed[fedP] = []
                    scoresFed[fedP] = []
                    diffsFed[fedP] = []
                    scoresStochFed[fedP] = []

                jsonDictKeyFed = str((convN, alpha, discount, epsilon, fedP, syncBackups, stochasticPolicy, envSeed))
                if jsonDictKeyFed in jsonDict:
#                     print(jsonDictKeyFed)
                    out_dict = jsonDict[jsonDictKeyFed]
                    epsFed[fedP].append(out_dict["episodes"])
                    scoresFed[fedP].append(out_dict["endScore"]/100)
                    diffsFed[fedP].append(np.mean(out_dict["diffs"][-10:]))
                    scoresStochFed[fedP].append(out_dict["endScoreStoch"]/100)

            jsonDictKeySin = str((convN, alpha, discount, epsilon, 1, -1, stochasticPolicy, envSeed))
            if jsonDictKeySin in jsonDict:
#                 print(jsonDictKeySin)
            # print(jsonDictKeySin)
                out_dict = jsonDict[jsonDictKeySin]

                epsSin.append(out_dict["episodes"])
                scoresSin.append(out_dict["endScore"]/100)
                scoresBell.append(out_dict["endScoreBell"]/100)
                scoresRand.append(out_dict["endScoreRand"]/100)
                scoresStochSin.append(out_dict["endScoreStoch"] / 100)
                scoresStochBell.append(out_dict["endScoreStochBell"] / 100)
                diffsSin.append(np.mean(out_dict["diffs"][-10:]))

                alphas.append(alpha)

                alpha /= 2
                
        data[str((convN, discount, epsilon, syncBackups, stochasticPolicy, envSeed))] = (epsSin,
                    epsFed,  
                    scoresSin,  
                    scoresFed,  
                    scoresBell,  
                    scoresStochSin,  
                    scoresStochFed,  
                    scoresStochBell,  
                    scoresRand,  
                    diffsSin,  
                    diffsFed,  
                    alphas)
                
# print("data")
# for key in data.keys():
#     print(key)

def loadData(convN=convN, discount=discount, epsilon=epsilon, syncBackups=syncBackups, stochasticPolicy=stochasticPolicy, envSeed=envSeed):
    global epsSin
    global epsFed
    global scoresSin
    global scoresFed
    global scoresBell
    global scoresStochSin
    global scoresStochFed
    global scoresStochBell
    global scoresRand 
    global diffsSin
    global diffsFed 
    global alphas
    (epsSin,
    epsFed,  
    scoresSin,  
    scoresFed,  
    scoresBell,  
    scoresStochSin,  
    scoresStochFed,  
    scoresStochBell,  
    scoresRand,  
    diffsSin,  
    diffsFed,  
    alphas) = data[str((convN, discount, epsilon, syncBackups, stochasticPolicy, envSeed))]

This graph is a comparison of learned models to the bellman model. A higher number of agents and smaller step size results in the lowest error to bellman. This trend appears linear on a log log plot for a variety of different numbers of agents.

In [ ]:
plt.clf()
for i in range(len(epsilons)):
    epsilon = epsilons[i]
    loadData(epsilon=epsilon, syncBackups=1)
    for placehold2 in [0]:
        plt.subplot(1, 2, i+1)
        plt.tight_layout(pad=3)
        plotName = "Q learning Difference from Bellman with Eps " + str(epsilon)
        plt.plot(alphas, np.array(diffsSin) ** 2, marker="x", label="Single Agent")
#         print(diffsFed)
        for fedP in fedPs:
#             print(fedP)
            plt.plot(alphas, np.array(diffsFed[fedP]) ** 2, marker="x", label=str(fedP) + " Agent")
        plt.legend()
        plt.xlabel("alpha - step size")
        if i == 0:
            plt.ylabel("distance squared to bellman q values")

        plt.title(plotName.replace("_", " "))
        # plt.ylim(0, 4)
        
#         plt.yscale('log', base=2)
#         plt.xscale('log', base=2)
plt.show()       

# plot lines divided by 1 agent

3d version of above plot by using number of agents as an additional axis instead of different series. This appears to be a plane on a log log log plot.

In [ ]:
epsilon = 1
loadData(epsilon=epsilon, syncBackups=1)
plt.clf()
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter3D((alphas), ([1]*len(alphas)), np.array(diffsSin) ** 2, label="Single Agent")
for fedP in fedPs:
    ax.scatter3D((alphas), [fedP]*len(alphas), np.array(diffsFed[fedP]) ** 2, label=str(fedP) + " Agent")
ax.set_xlabel("alpha - step size")
ax.set_ylabel("p - Number of Agents")
ax.set_zlabel("distance squared to bellman q values")
plotName = "Q learning Difference from Bellman with Eps " + str(epsilon)
plt.title(plotName)
plt.show()  
# ax.legend()
# ax.set_xscale('log', base=2)
# ax.set_zscale('log', base=2)
# plt.show()

# invert to show inverse curve as linear

Test Reward when evaluated with a deterministic policy. Everything appears to be the same as bellman. This may present an issue with using reward to learn and get closer to bellman values for MDPs

In [ ]:
plt.clf()
for i in range(len(epsilons)):
    epsilon = epsilons[i]
    loadData(epsilon=epsilon, syncBackups=1)
    for placehold2 in [0]:
        plt.subplot(1, 2, i+1)
        plt.tight_layout(pad=3)
        plotName = "Q learning Score with Eps " + str(epsilon)
        plt.plot(alphas, scoresSin, marker="x", label="Single Agent")
        for fedP in fedPs:
            plt.plot(alphas, scoresFed[fedP], marker="x", label=str(fedP) + " Agent")
        plt.hlines(np.mean(scoresBell), 0, 1, label="bellman score")
        plt.hlines(np.mean(scoresRand), 0, 1, label="random policy score")
        plt.legend()
        plt.xlabel("alpha - step size")
        if i == 0:
            plt.ylabel("Test Reward")

        plt.title(plotName.replace("_", " "))
        # plt.ylim(0, 4)
        # plt.yscale('log', base=2)
        # plt.xscale('log', base=2)
plt.show()

In [ ]:
epsilon = 1
loadData(epsilon=epsilon, syncBackups=1)
plt.clf()
fig = plt.figure()
ax = plt.axes(projection='3d')
# print(scoresSin)
ax.scatter3D(alphas, [1]*len(alphas), scoresSin, label="Single Agent")
for fedP in fedPs:
    ax.scatter3D(alphas, [fedP]*len(alphas), scoresFed[fedP], label=str(fedP) + " Agent")
ax.set_xlabel("alpha - step size")
ax.set_ylabel("p - Number of Agents")
ax.set_zlabel("Test Reward")
ax.ticklabel_format(useOffset=False)
xx, yy = np.meshgrid(np.array(range(10))/10, range(10))
z = xx - xx + yy - yy
ax.plot_surface(xx, yy, z + np.mean(scoresBell), alpha=0.2, label="bellman score")
ax.plot_surface(xx, yy, z + np.mean(scoresRand), alpha=0.2, label="random policy score")
plt.title(plotName.replace("_", " "))
# ax.legend()
# ax.set_xscale('log', base=2)
# ax.set_zscale('log', base=2)
plt.show()

Test Reward when evaluated with a stochastic policy. This is the omly graph where epsilon 0 is noticably different than epsilon 1. Here the Q learning can get a higher score than the bellman Q values. Shows a rough increase in reward for more agents and smaller step sizes.

In [ ]:
plt.clf()
for i in range(len(epsilons)):
    epsilon = epsilons[i]
    loadData(epsilon=epsilon, syncBackups=1)
    for placehold2 in [0]:
        plt.subplot(1, 2, i+1)
        plt.tight_layout(pad=3)
        plotName = "Q learning Score Stochastic Policy with Eps " + str(epsilon)
        plt.plot(alphas, scoresStochSin, marker="x", label="Single Agent")
        for fedP in fedPs:
            plt.plot(alphas, scoresStochFed[fedP], marker="x", label=str(fedP) + " Agent")
        plt.hlines(np.mean(scoresStochBell), 0, 1, label="bellman score")
        plt.hlines(np.mean(scoresRand), 0, 1, label="random policy score")
        plt.legend()
        plt.xlabel("alpha - step size")
        if i == 0:
            plt.ylabel("Test Reward")

        plt.title(plotName.replace("_", " "))
        # plt.ylim(0, 4)
        # plt.yscale('log', base=2, base=2)
        # plt.xscale('log', base=2, base=2)
plt.show()

In [ ]:
epsilon = 1
loadData(epsilon=epsilon, syncBackups=1)
plt.clf()
fig = plt.figure()
ax = plt.axes(projection='3d')
# print(scoresSin)
ax.scatter3D(alphas, [1]*len(alphas), scoresStochSin, label="Single Agent")
for fedP in fedPs:
    ax.scatter3D(alphas, [fedP]*len(alphas), scoresStochFed[fedP], label=str(fedP) + " Agent")
ax.set_xlabel("alpha - step size")
ax.set_ylabel("p - Number of Agents")
ax.set_zlabel("Test Reward")
ax.ticklabel_format(useOffset=False)
xx, yy = np.meshgrid(np.array(range(10))/10, range(10))
z = xx - xx + yy - yy
ax.plot_surface(xx, yy, z + np.mean(scoresStochBell), alpha=0.2, label="bellman score")
ax.plot_surface(xx, yy, z + np.mean(scoresRand), alpha=0.2, label="random policy score")
plt.title(plotName.replace("_", " "))
# ax.legend()
# ax.set_xscale('log', base=2)
# ax.set_zscale('log', base=2)
plt.show()

Convergence time, measured in model backups for a single agent (not all agents). This appears to show no speedup

In [ ]:
plt.clf()
for i in range(len(epsilons)):
    epsilon = epsilons[i]
    loadData(epsilon=epsilon, syncBackups=1)
    for placehold2 in [0]:
        plt.subplot(1, 2, i+1)
        plt.tight_layout(pad=4)
        plotName = "Q learning Convergence Time with Eps " + str(epsilon)
        plt.plot(alphas, np.array(epsSin) * 100, marker="x", label="Single Agent")
        for fedP in fedPs:
            plt.plot(alphas, np.array(epsFed[fedP]) * 100, marker="x", label=str(fedP) + " Agent")
        plt.legend()
        plt.xlabel("alpha - step size")
        if i == 0:
            plt.ylabel("Number of backups for a single agent")

        plt.title(plotName.replace("_", " "))
        # plt.ylim(0, 4)
        plt.yscale('log', base=2)
        plt.xscale('log', base=2)
plt.show()

In [ ]:
epsilon = 1
loadData(epsilon=epsilon, syncBackups=1)
plt.clf()
fig = plt.figure()
ax = plt.axes(projection='3d')
ax.scatter3D(alphas, [1]*len(alphas), epsSin, label="Single Agent")
for fedP in fedPs:
    ax.scatter3D(alphas, [fedP]*len(alphas), epsFed[fedP], label=str(fedP) + " Agent")
ax.set_xlabel("alpha - step size")
ax.set_ylabel("p - Number of Agents")
ax.set_zlabel("Number of backups for a single agent")
ax.ticklabel_format(useOffset=False)
plt.title(plotName.replace("_", " "))
# ax.legend()
# ax.set_xscale('log', base=2)
# ax.set_zscale('log', base=2)
plt.show()

In [ ]:
os.system("jupyter nbconvert --no-input --to pdf graphing.ipynb")

In [ ]:
plt.clf()

for i in range(len(fedPs)):
    for syncBackups in syncBackupValues:
        fedP = fedPs[i]
        epsilon = 1
#         syncBackups = 1
        loadData(epsilon=epsilon, syncBackups=syncBackups)
        for placehold2 in [0]:
            plt.subplot(2, len(fedPs)//2 + 1, i+1)
            plt.tight_layout(pad=3)
            plotName = "Distance to Bellman Q function with " + str(fedP) + " Agents"

            plt.plot(alphas, np.array(diffsFed[fedP]) ** 2, marker="x", label="K " + str(syncBackups))

            plt.legend()
            plt.xlabel("alpha - step size")
            if i == 0:
                plt.ylabel("final distance to bellman q values")

            plt.title(plotName.replace("_", " "))
            # plt.ylim(0, 4)
#             plt.yscale('log', base=2)
#             plt.xscale('log', base=2)
            
plt.rcParams['figure.figsize'] = [20, 20]
plt.show()

In [ ]:
plt.clf()
for i in range(len(fedPs)):
    for syncBackups in syncBackupValues:
        fedP = fedPs[i]
        epsilon = 1
#         syncBackups = 1
        loadData(epsilon=epsilon, syncBackups=syncBackups)
        for placehold2 in [0]:
            plt.subplot(2, len(fedPs)//2 + 1, i+1)
            plt.tight_layout(pad=3)
            plotName = "Q learning Convergence Time with " + str(fedP) + " Agents"

            plt.plot(alphas, np.array(epsFed[fedP]) * 100, marker="x", label="K " + str(syncBackups))

            plt.legend()
            plt.xlabel("alpha - step size")
            if i == 0:
                plt.ylabel("Number of backups for a single agent")

            plt.title(plotName.replace("_", " "))
            # plt.ylim(0, 4)
            
            plt.yscale('log', base=2)
            plt.xscale('log', base=2)
            
plt.rcParams['figure.figsize'] = [15, 15]
plt.show()

In [ ]:
# epsilon = 1
# loadData(epsilon=epsilon, syncBackups=1)
# plt.clf()
# fig = plt.figure()
# ax = plt.axes(projection='3d')
# ax.scatter3D(np.log2(alphas), np.log2([1]*len(alphas)), np.log2(diffsSin), label="Single Agent")
# for fedP in fedPs:
#     ax.scatter3D(np.log2(alphas), np.log2([fedP]*len(alphas)), np.log2(diffsFed[fedP]), label=str(fedP) + " Agent")
# ax.set_xlabel("alpha - step size log2")
# ax.set_ylabel("p - Number of Agents log2")
# ax.set_zlabel("final distance to bellman q values log2")
# plotName = "Q learning Difference from Bellman with Eps " + str(epsilon)
# plt.title(plotName)
# plt.show()  